In [1]:
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBRegressor
from xgboost import plot_tree #requieres graphviz https://www.graphviz.org/
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
pd.set_option('display.max_columns',None)

In [2]:
df_clean = pd.read_excel("C:/Users/Daniel/OneDrive - Universidad de Antioquia/Analitica y Ciencia de Datos/1/Seminario/Monografia/clean_5.xlsx")

# df_clean.rename(columns={
#     'CURRENT_A_CALC':,
#  'CURRENT_B_CALC':,
#  'CURRENT_C_CALC':,
#  'CURRENT_D_CALC':,
#  'POWER_A':,
#  'POWER_B':,
#  'POWER_C':,
#  'POWER_D':,
#  'TEMPERATURE_A':,
#  'TEMPERATURE_B':,
#  'TEMPERATURE_C':,
#  'TEMPERATURE_D':,
#  'VOLTAGE_A':,
#  'VOLTAGE_B':,
#  'VOLTAGE_C':,
#  'VOLTAGE_D':,
# },inplace=True)
# df_clean['ciclo'] = df_clean.groupby(['bateria','fecha_exp'])['fecha_exp'].cumcount()
##df_clean.groupby('bateria')['ciclo'].describe()

def contar(data):
    cuenta = 1
    cuentas = []
    for i in range(0,(len(data))):
        if i>0:
            if data['bateria'][i]!=data['bateria'][i-1]:
                cuenta = 1
            elif data['fecha_exp'][i]!=data['fecha_exp'][i-1]:
                cuenta = cuenta + 1
        cuentas.append(cuenta)
    return cuentas
df_clean['ciclo'] = contar(df_clean)

lista_metroplus = df_clean[df_clean['lugar']=='metroplus']['fecha_exp'].unique()

In [3]:
variables_ciclo_inicial = [
    'distancia', 'vel_max', 'vel_prom',
    'ACCELERATION_X', 'ACCELERATION_Y', 'ACCELERATION_Z', 'CURRENT_A_CALC',
    'CURRENT_B_CALC', 'CURRENT_C_CALC', 'CURRENT_D_CALC', 'POWER_A',
    'POWER_B', 'POWER_C', 'POWER_D', 'TEMPERATURE_A', 'TEMPERATURE_B',
    'TEMPERATURE_C', 'TEMPERATURE_D', 'VOLTAGE_A', 'VOLTAGE_B', 'VOLTAGE_C',
    'VOLTAGE_D', 'ENV_HUMIDITY', 'ENV_TEMPERATURE','ciclo','bateria'
    ]
variables_ciclo = [x for x in variables_ciclo_inicial if x != 'bateria']

variables_sin_ciclo_inicial = [
    'distancia', 'vel_max', 'vel_prom',
    'ACCELERATION_X', 'ACCELERATION_Y', 'ACCELERATION_Z', 'CURRENT_A_CALC',
    'CURRENT_B_CALC', 'CURRENT_C_CALC', 'CURRENT_D_CALC', 'POWER_A',
    'POWER_B', 'POWER_C', 'POWER_D', 'TEMPERATURE_A', 'TEMPERATURE_B',
    'TEMPERATURE_C', 'TEMPERATURE_D', 'VOLTAGE_A', 'VOLTAGE_B', 'VOLTAGE_C',
    'VOLTAGE_D', 'ENV_HUMIDITY', 'ENV_TEMPERATURE','bateria'
    ]
variables_sin_ciclo = [x for x in variables_sin_ciclo_inicial if x != 'bateria']

# ciclo

In [4]:
X = df_clean[df_clean['fecha_exp'].isin(lista_metroplus)][variables_ciclo_inicial]#[['CURRENT_A_CALC','POWER_A','TEMPERATURE_A','VOLTAGE_A']]
y = df_clean[df_clean['fecha_exp'].isin(lista_metroplus)]['RUL_registro']
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Se escalan los valores del dataset entrenamiento y prueba de las columnas numéricas
X_Min_Max=X.copy(deep=True)
X_Min_Max[X.columns] = scaler.fit_transform(X)
# X_test[num_vars] = scaler.fit_transform(X_test[num_vars])

# Se crean los datasets de entrenamiento y prueba para las variables de entrada y salida
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
                                        X,#X_Min_Max,
                                        y.values.reshape(-1),
                                        train_size   = 0.8,
                                        random_state = 123,
                                        shuffle      = True,
                                        stratify = X['bateria']
                                    )
X_train.drop('bateria',axis=1,inplace=True)
X_test.drop('bateria',axis=1,inplace=True)

In [5]:
X_train

,distancia,vel_max,vel_prom,ACCELERATION_X,ACCELERATION_Y,ACCELERATION_Z,CURRENT_A_CALC,CURRENT_B_CALC,CURRENT_C_CALC,CURRENT_D_CALC,POWER_A,POWER_B,POWER_C,POWER_D,TEMPERATURE_A,TEMPERATURE_B,TEMPERATURE_C,TEMPERATURE_D,VOLTAGE_A,VOLTAGE_B,VOLTAGE_C,VOLTAGE_D,ENV_HUMIDITY,ENV_TEMPERATURE,ciclo
26329,0.00,0.0,0.0,10.76,2.65,4.29,3.150,13.260000,12.640,14.920,0.09,0.69,0.65,0.77,41.095455,40.150000,40.150000,40.980,35.25,17.02,17.06,17.040,67.5500,22.600000,369
38621,15.73,60.4,17.1,7.64,2.65,1.25,1.445,6.906667,6.510,6.030,0.04,0.33,0.34,0.31,63.590000,62.939244,64.130000,64.130,35.45,17.27,17.19,17.280,67.8100,23.120000,800
62844,15.09,46.8,17.6,10.06,1.56,3.43,4.300,18.380000,16.630,18.190,0.12,0.95,0.86,0.94,23.442857,23.158571,23.440000,23.120,36.99,18.13,18.00,18.020,78.5500,22.338400,93
57287,15.46,42.5,19.1,9.98,0.31,3.90,1.500,6.830000,6.370,6.140,0.04,0.35,0.35,0.32,32.147923,32.460270,32.180435,32.290,38.27,18.66,18.57,18.660,42.4164,29.655918,178
35979,16.54,52.7,16.9,9.75,1.87,2.26,0.910,10.490000,13.980,13.220,0.03,0.54,0.72,0.69,97.310000,94.566486,97.310000,100.410,30.60,14.93,14.86,14.940,57.1300,23.730000,1266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59117,15.46,47.4,18.7,7.88,4.60,-2.03,0.550,0.340000,0.430,0.295,0.01,0.02,0.02,0.02,36.880000,36.766667,36.710000,36.768,38.06,18.54,18.44,18.570,67.7200,22.970000,725
27541,0.00,0.0,0.0,7.41,1.64,-1.72,0.330,0.380000,0.280,0.230,0.01,0.02,0.01,0.01,91.243846,89.671818,91.890000,91.890,31.39,15.36,15.25,15.360,68.4400,22.460000,1581
67528,15.22,46.0,17.7,10.30,1.79,1.17,2.020,8.140000,7.940,8.340,0.06,0.42,0.41,0.43,38.600000,38.427200,38.264000,38.600,34.86,17.09,16.98,17.065,65.2500,24.880000,661
22721,16.68,0.0,0.0,10.76,2.03,3.59,1.330,6.750000,6.660,7.070,0.04,0.35,0.35,0.37,35.014118,35.254000,34.940000,35.110,37.52,18.31,18.22,18.260,66.8500,23.670000,208


In [5]:
%%time
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=500,n_jobs=-1)
rfr.fit(X_train, y_train)

y_pred = rfr.predict(X_test)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred)*100)

32.64707858617726
99.44857245334036
CPU times: total: 13min 53s
Wall time: 46 s


In [7]:
ruta_tablas = 'C:/Users/Daniel/OneDrive - Universidad de Antioquia/Analitica y Ciencia de Datos/1/Seminario/Monografia/Tablas/'

In [8]:
imp_features = rfr.feature_importances_
# for i in imp_features:
    # print(i.round(3))
df_imp_features = pd.DataFrame({"features":variables_ciclo}).join(pd.DataFrame({"weights":imp_features}))
df_imp_features.sort_values(by=['weights'], ascending=False).to_excel(ruta_tablas+'rfr_importances_ciclo.xlsx',index=False)
df_imp_features.sort_values(by=['weights'], ascending=False)

,features,weights
18,VOLTAGE_A,0.186637
19,VOLTAGE_B,0.162261
14,TEMPERATURE_A,0.152055
20,VOLTAGE_C,0.116037
0,distancia,0.078468
24,ciclo,0.059886
2,vel_prom,0.048667
21,VOLTAGE_D,0.047790
15,TEMPERATURE_B,0.031311
16,TEMPERATURE_C,0.025807


In [15]:
%%time
from sklearn.svm import SVR

svr = SVR(C=100, degree=2, gamma=0.01,kernel='poly')
svr.fit(X_train, y_train)

y_pred = svr.predict(X_test)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred)*100)

In [9]:
%%time
gbr = GradientBoostingRegressor(learning_rate=0.2,max_depth=7,n_estimators=220)
gbr.fit(X_train, y_train)

y_pred = gbr.predict(X_test)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred)*100)

38.73570933713944
99.22371206788164
CPU times: total: 32.8 s
Wall time: 1min 14s


In [10]:
imp_features = gbr.feature_importances_
# for i in imp_features:
    # print(i.round(3))
df_imp_features = pd.DataFrame({"features":variables_ciclo}).join(pd.DataFrame({"weights":imp_features}))
df_imp_features.sort_values(by=['weights'], ascending=False).to_excel(ruta_tablas+'gbr_importances_ciclo.xlsx',index=False)
df_imp_features.sort_values(by=['weights'], ascending=False)

,features,weights
18,VOLTAGE_A,0.231105
19,VOLTAGE_B,0.173965
14,TEMPERATURE_A,0.095044
21,VOLTAGE_D,0.084493
0,distancia,0.078691
16,TEMPERATURE_C,0.050548
24,ciclo,0.049727
2,vel_prom,0.048823
20,VOLTAGE_C,0.038760
15,TEMPERATURE_B,0.034829


In [11]:
%%time
abr = AdaBoostRegressor(learning_rate=0.9,loss='square',n_estimators=180)
abr.fit(X_train, y_train)

y_pred = abr.predict(X_test)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(np.sqrt(mean_squared_error(y_test, y_pred,squared=False)))
print(r2_score(y_test, y_pred)*100)

12.892915715907725
85.70433433740976
CPU times: total: 5.8 s
Wall time: 13.2 s


In [11]:
print(np.sqrt(mean_squared_error(y_test, y_pred,squared=True)))
print(r2_score(y_test, y_pred)*100)

172.4894664924555
84.60693986054156


In [12]:
imp_features = abr.feature_importances_
# for i in imp_features:
    # print(i.round(3))
df_imp_features = pd.DataFrame({"features":variables_ciclo}).join(pd.DataFrame({"weights":imp_features}))
df_imp_features.sort_values(by=['weights'], ascending=False).to_excel(ruta_tablas+'abr_importances_ciclo.xlsx',index=False)
df_imp_features.sort_values(by=['weights'], ascending=False)

,features,weights
2,vel_prom,1.699221e-01
0,distancia,9.694007e-02
24,ciclo,9.355562e-02
18,VOLTAGE_A,8.638858e-02
15,TEMPERATURE_B,8.404679e-02
14,TEMPERATURE_A,8.217545e-02
21,VOLTAGE_D,5.589782e-02
20,VOLTAGE_C,5.533145e-02
19,VOLTAGE_B,4.729979e-02
16,TEMPERATURE_C,3.699993e-02


# Sin ciclo

In [13]:
X = df_clean[df_clean['fecha_exp'].isin(lista_metroplus)][variables_sin_ciclo_inicial]#[['CURRENT_A_CALC','POWER_A','TEMPERATURE_A','VOLTAGE_A']]
y = df_clean[df_clean['fecha_exp'].isin(lista_metroplus)]['RUL_registro']
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Se escalan los valores del dataset entrenamiento y prueba de las columnas numéricas
X_Min_Max=X.copy(deep=True)
X_Min_Max[X.columns] = scaler.fit_transform(X)
# X_test[num_vars] = scaler.fit_transform(X_test[num_vars])

# Se crean los datasets de entrenamiento y prueba para las variables de entrada y salida
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
                                        X,#X_Min_Max,
                                        y.values.reshape(-1),
                                        train_size   = 0.8,
                                        random_state = 123,
                                        shuffle      = True,
                                        stratify = X['bateria']
                                    )
X_train.drop('bateria',axis=1,inplace=True)
X_test.drop('bateria',axis=1,inplace=True)

In [14]:
%%time
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor(n_estimators=500,n_jobs=18)
rfr.fit(X_train, y_train)

y_pred = rfr.predict(X_test)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred)*100)

32.242282379815194
99.46216214926375
CPU times: total: 10min 6s
Wall time: 47.4 s


In [15]:
imp_features = rfr.feature_importances_
# for i in imp_features:
    # print(i.round(3))
df_imp_features = pd.DataFrame({"features":variables_ciclo}).join(pd.DataFrame({"weights":imp_features}))
df_imp_features.sort_values(by=['weights'], ascending=False).to_excel(ruta_tablas+'rfr_importances_sin_ciclo.xlsx',index=False)
df_imp_features.sort_values(by=['weights'], ascending=False)

,features,weights
18,VOLTAGE_A,0.185981
19,VOLTAGE_B,0.165729
14,TEMPERATURE_A,0.151364
20,VOLTAGE_C,0.122426
0,distancia,0.094242
2,vel_prom,0.064640
21,VOLTAGE_D,0.034258
15,TEMPERATURE_B,0.028648
23,ENV_TEMPERATURE,0.028403
22,ENV_HUMIDITY,0.028198


In [16]:
%%time
gbr = GradientBoostingRegressor(learning_rate=0.2,max_depth=7,n_estimators=220)
gbr.fit(X_train, y_train)

y_pred = gbr.predict(X_test)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred)*100)

38.76836430602351
99.2224026640352
CPU times: total: 36.7 s
Wall time: 1min 12s


In [17]:
imp_features = gbr.feature_importances_
# for i in imp_features:
    # print(i.round(3))
df_imp_features = pd.DataFrame({"features":variables_ciclo}).join(pd.DataFrame({"weights":imp_features}))
df_imp_features.sort_values(by=['weights'], ascending=False).to_excel(ruta_tablas+'gbr_importances_sin_ciclo.xlsx',index=False)
df_imp_features.sort_values(by=['weights'], ascending=False)

,features,weights
18,VOLTAGE_A,0.267517
19,VOLTAGE_B,0.169108
0,distancia,0.098960
14,TEMPERATURE_A,0.093412
21,VOLTAGE_D,0.067615
2,vel_prom,0.055176
16,TEMPERATURE_C,0.047713
15,TEMPERATURE_B,0.043803
20,VOLTAGE_C,0.023243
22,ENV_HUMIDITY,0.021467


In [18]:
%%time
abr = AdaBoostRegressor(learning_rate=0.9,loss='square',n_estimators=130)
abr.fit(X_train, y_train)

y_pred = abr.predict(X_test)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred)*100)

184.50214357981937
82.38824477804833
CPU times: total: 4.98 s
Wall time: 9.95 s


In [19]:
imp_features = abr.feature_importances_
# for i in imp_features:
    # print(i.round(3))
df_imp_features = pd.DataFrame({"features":variables_ciclo}).join(pd.DataFrame({"weights":imp_features}))
df_imp_features.sort_values(by=['weights'], ascending=False).to_excel(ruta_tablas+'abr_importances_sin_ciclo.xlsx',index=False)